## 8.2 DeepDream
### 8.2.1 用Keras实现DeepDream
**代码清单 8-8** 加载预训练的Inception V3模型

In [ ]:
from keras.applications import inception_v3
from keras import backend as K

# 防止Tensorflow运行GPU内存不足造成错误
import os
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = "0"  #选择哪一块gpu
config = ConfigProto()
config.allow_soft_placement=True #如果你指定的设备不存在，允许TF自动分配设备
config.gpu_options.per_process_gpu_memory_fraction=0.3  #分配百分之七十的显存给程序使用，避免内存溢出，可以自己调整
config.gpu_options.allow_growth = True   #按需分配显存，这个比较重要
session = InteractiveSession(config=config)

# 不需要训练模型
# 禁用所有与训练有关的操作
K.set_learning_phase(0)

# 构建不包括全连接层的Inception V3网络
# 使用预训练的ImageNet权重
model = inception_v3.InceptionV3(weights='imagenet',
                                 include_top=False)

**代码清单 8-9** 设置DeepDream配置

In [ ]:
# 这个字典将层的名称映射为一个系数，这个系数定量表示
# 该层激活对要最大化的损失函数的贡献大小。注意：层的
# 名称硬编码在内置的Inception V3应用中。可以使用
# model.summary()列出所有层的名称。
layer_contributions = {
    'mixed2': 0.2,
    'mixed3': 3.,
    'mixed4': 2.,
    'mixed5': 1.5,
}

**代码清单 8-10** 定义需要最大化的损失

In [ ]:
# 创建一个字典，将层的名称映射为层的实例
layer_dict = dict([(layer.name, layer) for layer in model.layers])

# 在定义损失时将层的贡献添加到这个标量变量中
loss = K.variable(0.)
for layer_name in layer_contributions:
    # 将该层特征的L2范数添加到loss中
    coeff = layer_contributions[layer_name]
    # 获取层的输出
    activation = layer_dict[layer_name].output

    # 为了避免出现边界伪影，损失中仅包含非边界的像素
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss = loss + coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

**代码清单 8-11** 梯度上升过程

In [ ]:
# 这个张量用于保存生成的图像，即梦境图像
dream = model.input

# 计算损失相对于梦境图像的梯度
grads = K.gradients(loss, dream)[0]

# 将梯度标准化（重要技巧）
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

# 给定一张输出图像，设置一个Keras函数来获取损失值和梯度值
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...Loss value at', i, ':', loss_value)
        x += step * grad_values
    return x

**代码清单 8-13** 辅助函数

In [ ]:
import imageio
from keras.preprocessing import image

def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
               float(size[0]) / img.shape[1],
               float(size[1]) / img.shape[2],
               1)
    return scipy.ndimage.zoom(img, factors, order=1)


def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    imageio.imwrite(fname, pil_img)


def preprocess_image(image_path):
    # 通用函数，用于打开图像=改变图像大小以及将
    # 图像格式转换为inception V3模型能够处理的张量
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img


def deprocess_image(x):
    # 通用函数，将一个张量转换为有效图像
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

**代码清单 8-12** 在多个连续尺度上运行梯度上升

In [ ]:
import numpy as np

# 改变这些超参数，可以得到新的效果

step = 0.01  # 梯度上升步长
num_octave = 3  # 运行梯度上升的尺度个数
octave_scale = 1.4  # 两个尺度之间大小比例
iterations = 20  # 在每个尺度上运行梯度上升的步数

# 如果损失增大到大于 10,
# 中断梯度上升过程，以避免得到丑陋的伪影
max_loss = 10.

base_image_path = './original_photo_deep_dream.jpg'

# 加载基础图像为Numpy数组（函数定义在代码清单8-13）
img = preprocess_image(base_image_path)

# 准备一个由形状元组组成的列表
# 它定义了运行梯度上升的不同尺度
original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)

# 反转为升序
successive_shapes = successive_shapes[::-1]

# 将Numpy数组缩到最小尺寸
original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print('Processing image shape', shape)
    # 放大梦境图像
    img = resize_img(img, shape)
    img = gradient_ascent(img,
                          iterations=iterations,
                          step=step,
                          max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img

    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale_' + str(shape) + '.png')

save_img(img, fname='final_dream.png')